In [1]:
import torch
import flwr as fl
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, Normalize, ToTensor
from client import get_client_generator, weighted_average_accuracy
from dataset import partition_dataset
from util import seed_everything
from flwr.server.strategy import FedAvg

transform = Compose([ToTensor(), Normalize((0.1307,), (0.3081,))])
train_dataset = MNIST("./mnist", train=True, download=True, transform=transform)
val_dataset = MNIST("./mnist", train=False, transform=transform)

def complete_run(seed=None):
    if seed is not None:
        seed_everything(seed)

    num_clients = 10
    train_datasets = partition_dataset(train_dataset, num_clients)
    val_datasets = partition_dataset(val_dataset, num_clients)
    train_dataloaders = [torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True) for dataset in train_datasets]
    val_dataloaders = [torch.utils.data.DataLoader(dataset, batch_size=16) for dataset in val_datasets]
    client_resources = None
    client_fn = get_client_generator(train_dataloaders, val_dataloaders)
    client_config = {
        "lr": 0.05,
        "epochs": 1,
    }
    strategy = FedAvg(
        min_fit_clients=2,
        min_evaluate_clients=2,
        min_available_clients=2,
        fraction_fit=0.5,
        fraction_evaluate=0.5,
        on_fit_config_fn=lambda _: client_config,
        on_evaluate_config_fn=lambda _: client_config,
        evaluate_metrics_aggregation_fn=weighted_average_accuracy,
    )
    hist = fl.simulation.start_simulation(
        client_fn=client_fn,
        seed_fn=seed_everything if seed is not None else None,
        seed=seed,
        num_clients=num_clients,
        config=fl.server.ServerConfig(num_rounds=2),
        client_resources=client_resources,
        strategy=strategy
    )
    return hist

In [2]:
run1 = complete_run(0)

INFO flwr 2023-03-08 16:28:26,587 | app.py:176 | Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
2023-03-08 16:28:28,617	INFO worker.py:1553 -- Started a local Ray instance.
INFO flwr 2023-03-08 16:28:30,165 | app.py:210 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'memory': 17053086516.0, 'CPU': 10.0, 'object_store_memory': 2147483648.0}
INFO flwr 2023-03-08 16:28:30,167 | server.py:100 | Initializing global parameters
INFO flwr 2023-03-08 16:28:30,167 | server.py:293 | Requesting initial parameters from one random client
INFO flwr 2023-03-08 16:28:30,842 | server.py:297 | Received initial parameters from one random client
INFO flwr 2023-03-08 16:28:30,843 | server.py:102 | Evaluating initial parameters
INFO flwr 2023-03-08 16:28:30,843 | server.py:115 | FL starting
DEBUG flwr 2023-03-08 16:28:30,843 | server.py:234 | fit_round 1: strategy sampled 5 clients (out of 10)
DEBUG flwr 2023-03-08 16:28:39,930 | server.py:248 | fit_

In [3]:
run2 = complete_run(0)

INFO flwr 2023-03-08 16:29:18,203 | app.py:176 | Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
2023-03-08 16:29:22,062	INFO worker.py:1553 -- Started a local Ray instance.
INFO flwr 2023-03-08 16:29:23,560 | app.py:210 | Flower VCE: Ray initialized with resources: {'memory': 17183496602.0, 'node:127.0.0.1': 1.0, 'CPU': 10.0, 'object_store_memory': 2147483648.0}
INFO flwr 2023-03-08 16:29:23,561 | server.py:100 | Initializing global parameters
INFO flwr 2023-03-08 16:29:23,562 | server.py:293 | Requesting initial parameters from one random client
INFO flwr 2023-03-08 16:29:24,239 | server.py:297 | Received initial parameters from one random client
INFO flwr 2023-03-08 16:29:24,239 | server.py:102 | Evaluating initial parameters
INFO flwr 2023-03-08 16:29:24,240 | server.py:115 | FL starting
DEBUG flwr 2023-03-08 16:29:24,240 | server.py:234 | fit_round 1: strategy sampled 5 clients (out of 10)
DEBUG flwr 2023-03-08 16:29:33,248 | server.py:248 | fit_

In [4]:
run1

History (loss, distributed):
	round 1: 0.341917051410675
	round 2: 0.2334419816419482
	round 3: 0.1811397484153509
	round 4: 0.1350405248582363
	round 5: 0.1206499027621001
History (metrics, distributed):
{'accuracy': [(1, 0.9072), (2, 0.9336), (3, 0.9464), (4, 0.9584), (5, 0.9646)]}

In [5]:
run2

History (loss, distributed):
	round 1: 0.341917051410675
	round 2: 0.2334419816419482
	round 3: 0.1811397484153509
	round 4: 0.1350405248582363
	round 5: 0.1206499027621001
History (metrics, distributed):
{'accuracy': [(1, 0.9072), (2, 0.9336), (3, 0.9464), (4, 0.9584), (5, 0.9646)]}

In [7]:
run3 = complete_run()

INFO flwr 2023-03-08 16:31:31,400 | app.py:176 | Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
2023-03-08 16:31:35,613	INFO worker.py:1553 -- Started a local Ray instance.
INFO flwr 2023-03-08 16:31:37,089 | app.py:210 | Flower VCE: Ray initialized with resources: {'memory': 17119176295.0, 'CPU': 10.0, 'object_store_memory': 2147483648.0, 'node:127.0.0.1': 1.0}
INFO flwr 2023-03-08 16:31:37,090 | server.py:100 | Initializing global parameters
INFO flwr 2023-03-08 16:31:37,090 | server.py:293 | Requesting initial parameters from one random client
INFO flwr 2023-03-08 16:31:37,724 | server.py:297 | Received initial parameters from one random client
INFO flwr 2023-03-08 16:31:37,724 | server.py:102 | Evaluating initial parameters
INFO flwr 2023-03-08 16:31:37,724 | server.py:115 | FL starting
DEBUG flwr 2023-03-08 16:31:37,725 | server.py:234 | fit_round 1: strategy sampled 5 clients (out of 10)
DEBUG flwr 2023-03-08 16:31:46,756 | server.py:248 | fit_